In [113]:

import nltk
import numpy as np
from textblob import TextBlob
import pandas as pd 
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import KFold, StratifiedKFold
#Load libraries needed for classification 
from sklearn import naive_bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.naive_bayes import BernoulliNB, MultinomialNB,GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import ComplementNB
#from bayes.classifiers import ComplementNB
#from naiveBayesClassifier.classifier import Classifier
from sklearn import svm, datasets
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.util import ngrams
from nltk.corpus import wordnet as omw # open multilingual wordnet
from nltk.stem.isri import ISRIStemmer

In [114]:
# A.read the course names training data & apply preprocessing into a data frame
# A.1 upload  Train data file names
path =r'D:\MahmoudOut\Final Project\CoursesAnalysis-master'
ds1_train=pd.read_excel(path+"\TrainData.xlsx")

In [115]:
#get Arabic sentences only
import re

def ArabicOnly(text):
    #print (text)
    result = re.sub(r'[^ء-ي]',' ',str(text)) #arabic
    if len(result.strip()) >0:
        return 'T'
    else:
        return 'F'
ds1_train['ArabicOnly'] =ds1_train['valueName'].apply(lambda x : ArabicOnly(x))


In [116]:
ds1_train = ds1_train[ds1_train.ArabicOnly =='T']
len(ds1_train)

79

In [117]:
# Arabic diacritics
arabic_punct = ''' ` ÷ × ؛ < > _ ( ) * & ^ % ] [ ـ ، / : " ؟ . , ' { } ~ ¦ + | !  ”  …  “ –   ـ  '''
arabic_diacritics = '''  َ     ُ       ِ      ّ       ً      ٌ       ٍ      ْ     '''

arabic_punct = arabic_punct.split()

arabic_diacritics = arabic_diacritics.split()

# Arabic punctuations and dicritis 
punctuations = set( arabic_punct  + arabic_diacritics )

# Arabic stopwords. This list are obtained from http://www.ranks.nl/stopwords/arabic 
asw = open(path+'/ArabicStopwords.txt', encoding="utf8").read()

arabicStopWords =  asw.split() #+ aswUinicode.split()

asw2 = ''

# Arabic  stopwords
all_stopwords = set(arabicStopWords)


In [118]:
all_stopwords

{'آخر',
 'أ',
 'أكثر',
 'ألا',
 'أما',
 'أن',
 'أنا',
 'أنت',
 'أنفسكم',
 'أنفسنا',
 'أنفسهم',
 'أنه',
 'أنها',
 'أو',
 'أى',
 'أي',
 'أيضا',
 'إذا',
 'إضافي',
 'إلا',
 'إلى',
 'إلي',
 'إما',
 'إن',
 'إنه',
 'إنها',
 'ا',
 'اذا',
 'اكثر',
 'الآن',
 'الا',
 'الان',
 'التى',
 'التي',
 'الذى',
 'الذي',
 'الذين',
 'الى',
 'الي',
 'اما',
 'ان',
 'انت',
 'انه',
 'انها',
 'او',
 'اى',
 'اي',
 'ايضا',
 'ب',
 'بأن',
 'بان',
 'بسبب',
 'بشكل',
 'بعد',
 'بعض',
 'بعيدا',
 'بن',
 'به',
 'بها',
 'بين',
 'بينما',
 "تحت'",
 'تكون',
 'تلك',
 'تم',
 'ثانية',
 'ثم',
 'جدا',
 'جميع',
 'حاليا',
 'حتى',
 'حول',
 'حيث',
 'حين',
 'خارج',
 'خلال',
 'دون',
 'ديك',
 'ذلك',
 'سوف',
 'صباحا',
 'ضد',
 'ضمن',
 'على',
 'عليه',
 'عليها',
 'عن',
 'عند',
 'عندما',
 'غير',
 'ف',
 'فأن',
 'فان',
 'فعل',
 'فقط',
 'فما',
 'فهو',
 'فهى',
 'فهي',
 'فوق',
 'فى',
 'في',
 'فيه',
 'فيها',
 'قبل',
 'قد',
 'قليل',
 'كان',
 'كانت',
 'كل',
 'كلا',
 'كما',
 'كيف',
 'ل',
 'لا',
 'لدى',
 'لديه',
 'لقاء',
 'لك',
 'لكن',
 'لم',
 'لماذا',
 

In [119]:
# remove punctcutions
def remove_punct(word):
    for c in word: return ''.join(ch for ch in word if not ch in punctuations) # remove punctuation
###################################################################################

In [120]:
# remove arabic diacritics
def remove_diacritics(text):
    arstemmer = ISRIStemmer()
    result = arstemmer.norm(text, num=1) #  remove diacritics which representing Arabic short vowels
    return result
###################################################################################

In [121]:
# takes a word list and returns the root for each Arabic words
def getRootAr(word_list):
    result = []
    arstemmer = ISRIStemmer()
    for word in word_list: result.append(arstemmer.stem(word))
    return result#' '.join(result)

In [122]:
# Arabic light stemming for Arabic text
# takes a word list and perform light stemming for each Arabic words
def lightStemAr(word_list):
    result = []
    arstemmer = ISRIStemmer()
    for word in word_list:
        word = arstemmer.norm(word, num=1)  #  remove diacritics which representing Arabic short vowels  
        if not word in arstemmer.stop_words:   # exclude stop words from being processed
            word = arstemmer.pre32(word)        # remove length three and length two prefixes in this order
            word = arstemmer.suf32(word)        # remove length three and length two suffixes in this order
            word = arstemmer.waw(word)          # remove connective ‘و’ if it precedes a word beginning with ‘و’
            word = arstemmer.norm(word, num=2)       # normalize initial hamza to bare alif
        result.append(word)
    return result#' '.join(result)

In [123]:
# takes a string of text and returns the word list (tonkized words)
# processing includes: removing diacritics and punctcutions, removing stopwords, and tokenizing
# Stemmning
# Get Root
def process_text(text, removePunct=True, removeSW=True, removeNum=True, Stem=True, Root=True):
    text = remove_diacritics(text)# remove arabic diacritics
    word_list = nltk.tokenize.wordpunct_tokenize(text.lower())
    if removePunct:
        word_list = [ w for w in word_list if not w in punctuations ]
        word_list = [ remove_punct(w) for w in word_list ]
    if removeSW: word_list = [ w for w in word_list if not w in all_stopwords ]
    if removeNum: word_list = [ w for w in word_list if not w.isdigit() ]
    if Stem: word_list = lightStemAr(word_list) #strimming
    if Root: word_list = getRootAr(word_list) #get Root
    return ' '.join(word_list)
###################################################################################

In [124]:
ds1_train['preprocessingWithStem'] =ds1_train['valueName'].apply(lambda x : process_text(x,True,True,False,True,False))

In [125]:
ds1_train['preprocessingWithRoot'] =ds1_train['valueName'].apply(lambda x : process_text(x,True,True,False,True,True))

In [126]:
ds1_train

,valueName,ArabicOnly,preprocessingWithStem,preprocessingWithRoot
43,دورة المعايير الدولية لإعداد التقارير المالية ...,T,دورة معايير دولية لإعداد تقارير مالية - ifrs,دور معايير دول إعداد قرر الة - ifrs
108,دورو معايير المحاسبة المصرية - 3 أجزاء,T,دورو معايير محاسبة مصرية - 3 اجزاء,دورو معايير حسب صرة - 3 جزء
109,دورة في المراجعة الداخلية بمفهومها الحديث,T,دورة مراجعة داخلية بمفهوم حديث,دور رجع دخل فهم حدث
110,دورة معايير المراجعة المصرية,T,دورة معايير مراجعة مصرية,دور معايير رجع صرة
111,دورة تدريبية في المعيار الدولي (IFRS 9),T,دورة تدريبية معيار دولي ifrs 9,دور درب عير دول ifrs 9
112,دورة تدريبية في المعيار الدولي (IFRS 15),T,دورة تدريبية معيار دولي ifrs 15,دور درب عير دول ifrs 15
113,دورة تدريبية في المعيار الدولي (IFRS 16),T,دورة تدريبية معيار دولي ifrs 16,دور درب عير دول ifrs 16
114,دورة الإعداد العلمي والفني للمحاسبين الجدد,T,دورة اعداد علمي فني محاسب جدد,دور عدد علم فني حسب جدد
115,دورة المعلومات المالية لغير الماليين,T,دورة معلوم مالية لغير مالي,دور علم الة لغر الي
116,البرنامج المتكامل في الضرائب,T,برنامج متكامل ضرائب,رنمج كامل ضرب
